In [1]:
import pandas as pd
import os
import time
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('gene2pubmed.gz', sep='\t') #20200103　downloaded

In [3]:
print(len(df))
df.head()

13035707


,#tax_id,GeneID,PubMed_ID
0,33,5961931,18245244
1,33,5961932,18245244
2,33,5961933,18245244
3,33,5961934,18245244
4,33,5961935,18245244


In [4]:
df_hs =df.loc[df['#tax_id']==9606,] 
print(len(df_hs))
df_hs.head()

1567493


,#tax_id,GeneID,PubMed_ID
2970546,9606,1,2591067
2970547,9606,1,3458201
2970548,9606,1,3610142
2970549,9606,1,8889549
2970550,9606,1,12477932


In [5]:
#Simpson
def sim_simpson(p, a, b):
    si = {}
    for item in p[a]:
        if item in p[b]:
            si[item] = 1
            
    n = len(si)
    if n == 0:
        return 0
    
    a_ = len(p[a])
    b_ = len(p[b])

    return n / min(a_, b_)

In [6]:
HIF1A = df_hs.loc[df_hs.GeneID==3091,].PubMed_ID.tolist() #HIF1A GeneID: 3091

In [7]:
gene_l = df_hs.GeneID.unique().tolist()
print(len(gene_l))
print(len(df_hs))
gene_l[:5]

38738
1567493


[1, 2, 3, 9, 10]

In [8]:
# i = 1246500
df2 = pd.DataFrame()
df3 = pd.DataFrame()
data = {}
data['HIF1A'] = HIF1A

for i in tqdm(gene_l):
    i_l = df_hs.loc[df.GeneID==i,].PubMed_ID.tolist()
    data['i_l'] = i_l
    df3['GeneID'] = [i]
    df3['simpson'] = [sim_simpson(data, 'HIF1A', 'i_l')]
    df2 = pd.concat([df2, df3])
    

  0%|          | 0/38738 [00:00<?, ?it/s]

In [9]:
df2.head()

,GeneID,simpson
0,1,0.129032
0,2,0.036765
0,3,0.166667
0,9,0.032653
0,10,0.012784


In [10]:
df2.describe()

,GeneID,simpson
count,3.873800e+04,38738.000000
mean,3.424948e+07,0.176976
std,4.939231e+07,0.251575
min,1.000000e+00,0.000000
25%,2.603725e+04,0.000000
50%,2.199845e+05,0.075269
75%,1.003022e+08,0.250000
max,1.198639e+08,1.000000


In [11]:
df2['simp_rnk'] = df2['simpson'].rank(ascending=False, method='min') 
df2.sort_values('simp_rnk').head()

,GeneID,simpson,simp_rnk
0,401667,1.0,1.0
0,100874044,1.0,1.0
0,100874047,1.0,1.0
0,100874056,1.0,1.0
0,100874058,1.0,1.0


In [12]:
humanGeneID = pd.read_csv('HumanGene_info.gz', sep='\t')
humanGeneID.head()

,#tax_id,GeneID,Symbol,Synonyms,description,Nomenclature_status
0,9606,1,A1BG,A1B|ABG|GAB|HYST2477,alpha-1-B glycoprotein,O
1,9606,2,A2M,A2MD|CPAMD5|FWP007|S863-7,alpha-2-macroglobulin,O
2,9606,3,A2MP1,A2MP,alpha-2-macroglobulin pseudogene 1,O
3,9606,9,NAT1,AAC1|MNAT|NAT-1|NATI,N-acetyltransferase 1,O
4,9606,10,NAT2,AAC2|NAT-2|PNAT,N-acetyltransferase 2,O


In [13]:
HIF1A_Simp = pd.merge(df2, humanGeneID[['GeneID', 'Symbol', 'description']], on='GeneID').rename(columns={'simpson': 'HIF1A_simpson', 'simp_rnk':'HIF1A_simpsonrnk'})
HIF1A_Simp.head()

,GeneID,HIF1A_simpson,HIF1A_simpsonrnk,Symbol,description
0,1,0.129032,15989.0,A1BG,alpha-1-B glycoprotein
1,2,0.036765,21741.0,A2M,alpha-2-macroglobulin
2,3,0.166667,13661.0,A2MP1,alpha-2-macroglobulin pseudogene 1
3,9,0.032653,21971.0,NAT1,N-acetyltransferase 1
4,10,0.012784,23036.0,NAT2,N-acetyltransferase 2


In [14]:
HIF1A_Simp.to_csv("G2P_HIF1A_Simpon.csv", index=False)